![image](car.jpeg)

**Car-ing is sharing**, an auto dealership company for car sales and rental, is taking their services to the next level thanks to **Large Language Models (LLMs)**.

As their newly recruited AI and NLP developer, you've been asked to prototype a chatbot app with multiple functionalities that not only assist customers but also provide support to human agents in the company.

The solution should receive textual prompts and use a variety of pre-trained Hugging Face LLMs to respond to a series of tasks, e.g. classifying the sentiment in a car’s text review, answering a customer question, summarizing or translating text, etc.


## Before you start

In order to complete the project you may wish to install some Hugging Face libraries such as `transformers` and `evaluate`.

In [10]:
!pip install transformers
!pip install evaluate
!pip install xformers
!pip install sacrebleu

from transformers import logging
logging.set_verbosity(logging.WARNING)

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


## Task 1

Use a pre-trained LLM to classify the sentiment of the five car reviews in the car_reviews dataset, and evaluate the classification accuracy and F1 score of predictions.

In [11]:
import numpy as np
import pandas as pd

# Import the data into a DataFrame
reviews = pd.read_csv('data/car_reviews.txt', delimiter=';')

# Change Positive/Negative to 1/0
reviews['labels'] = [1 if sentiment == 'POSITIVE' else 0 for sentiment in reviews['Class']]
reviews.drop('Class', axis=1, inplace=True)
reviews

,Review,labels
0,I am very satisfied with my 2014 Nissan NV SL....,1
1,The car is fine. Its a bit loud and not very p...,0
2,My first foreign car. Love it I would buy anot...,1
3,Ive come across numerous reviews praising the ...,0
4,Ive been dreaming of owning an SUV for quite a...,1


In [12]:
from transformers import pipeline

# Import a model tuned for sentiment analysis
model_name = 'distilbert-base-uncased-finetuned-sst-2-english'
sentiment_analysis = pipeline("sentiment-analysis", model=model_name)

# Generate predictions based on the reviews
predictions = sentiment_analysis(
  [example for example in reviews["Review"].to_list()]
)

# Convert to 1/0
predicted_labels = [1 if pred["label"] == "POSITIVE"
                    else 0 for pred in predictions]
predicted_labels

[1, 0, 1, 0, 1]

In [13]:
# Load the metrics and evaluate the results
from sklearn.metrics import accuracy_score, f1_score

accuracy_result = accuracy_score(reviews['labels'].to_list(), predicted_labels)
f1_result = f1_score(reviews['labels'].to_list(), predicted_labels)

print(f'Accuracy: {accuracy_result}')
print(f'F1 Score: {f1_result}')

Accuracy: 1.0
F1 Score: 1.0


All 5 reviews were classified correctly, giving an accuracy score and F1 score of 1.0.

## Task 2

Extract and pass the first two sentences of the first review in the dataset to an English-to-Spanish translation LLM. Calculate the BLEU score to assess translation quality, using the content in reference_translations.txt as references.

In [14]:
input_seq = "I am very satisfied with my 2014 Nissan NV SL. I use this van for my business deliveries and personal use."

# Load pretrained model for English to Spanish translation
model_name = "Helsinki-NLP/opus-mt-en-es"
translator = pipeline('translation', model=model_name)

# Translate first review
translated_review = translator(input_seq)[0]['translation_text']
print("Translation (Spanish):", translated_review)

Translation (Spanish): Estoy muy satisfecho con mi Nissan NV SL 2014. Uso esta camioneta para mis entregas de negocios y uso personal.


In [15]:
# Read the reference translations from the text file and import as a list
with open('data/reference_translations.txt', 'r', encoding='utf-8') as file:
    reference_translations = [line.strip() for line in file.readlines()]

reference_translations

['Estoy muy satisfecho con mi Nissan NV SL 2014. Utilizo esta camioneta para mis entregas comerciales y uso personal.',
 'Estoy muy satisfecho con mi Nissan NV SL 2014. Uso esta furgoneta para mis entregas comerciales y uso personal.']

In [16]:
import sacrebleu

bleu_score = sacrebleu.corpus_bleu([translated_review], [reference_translations]).score

print(bleu_score)

68.88074582865497


The model achieved a BLEU score of 68.9 for this translation.

## Task 3

The 2nd review in the dataset emphasizes brand aspects. Load an extractive QA LLM to formulate the question "What did he like about the brand?" and obtain an answer.

In [17]:
# New model for question answering
model_name = "deepset/minilm-uncased-squad2"
question_answerer = pipeline("question-answering", model=model_name)

# Provide question and context
question = 'What did he like about the brand?'
context = reviews['Review'][1]
answer = question_answerer(question=question, context=context)['answer']

# Print response
print(f'Q: What did he like about the brand?\nA: {answer}')

Q: What did he like about the brand?
A: ride quality reliability


## Task 4

Summarize the last review in the dataset, into approximately 50-55 tokens long. Store it in the variable summarized_text.

In [18]:
# Pretrained model for summarization
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
model_name = "t5-small"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize the input text
input_text = reviews['Review'][4]
inputs = tokenizer.encode("summarize: " + input_text, return_tensors="pt", max_length=512, truncation=True)

# Summarize and display
summary_ids = model.generate(inputs, max_length=55, min_length=50, length_penalty=2.0, num_beams=4, early_stopping=True)
summarized_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(summarized_text)

the Nissan Rogue provides me with the desired experience without burdening me with an exorbitant payment. I have hauled 12 bags of mulch in the back with the seats down and could have held more. the engine delivers strong performance, and the
